In [288]:
import json
import requests
import pandas as pd
import plotly.express as px
from dash import html, dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import numpy as np
import sqlite3
pd.options.mode.use_inf_as_na = True

In [334]:
init_loc = dict(
    lat = 52.5162829,
    lon = 13.3777240
)

In [267]:
def get_pagelist_around_location(lat, lon, radius=1000, gslimit=500):
    url = "https://de.wikipedia.org/w/api.php"
    query_params = {
        "action": "query",
        "format": "json",
        "list": "geosearch",
        "formatversion": "2",
        "gscoord": f"{str(lat)}|{str(lon)}",
        "gsradius": str(radius),
        "gslimit": str(gslimit) # results capped at 500 anyway for anon users
    }
    response = requests.get(url, params = query_params)
    response_dict = json.loads(response.text)
    out = pd.json_normalize(response_dict["query"]["geosearch"])
    out = out[ ["pageid", "title", "lat", "lon"] ]
    return( out.set_index("pageid") )



def query_views(ids, days = 30):
    url = "https://de.wikipedia.org/w/api.php"
    page_id_str = "|".join(map(str, ids[0:50]))
    query_params = {
        "action": "query",
        "format": "json",
        "prop": "pageviews",
        "pvipdays": str(days),
        "pageids": page_id_str,
        "formatversion": "2"
    }
    response = requests.get(url, params = query_params)
    response_dict = json.loads(response.text)
    response_df = pd.json_normalize(response_dict["query"]["pages"])
    views = response_df.iloc[:,0:3]
    views["views"] = response_df.filter(regex="pageviews").sum(axis=1)
    views.set_index("pageid", inplace=True)
    return(views["views"])



def shorten(ls, chunksize=50):
    if len(ls) >= chunksize:
        return( ls[50:len(ls)] )
    else:
        return([])
    
    

def get_viewcounts(ids, days=30):
    page_views = query_views(ids[0:50])
    ids = shorten(ids)

    while len(ids) > 0:
        chunk = query_views(ids)
        page_views = pd.concat( [page_views, chunk], axis=0 )
        ids = shorten(ids)
    return(page_views)



def opacity(selected):
    if selected:
        return 1.
    else:
        return .4



def histogram_df(df, column="log_views", bins=20):
    counts, boundaries = np.histogram(df[column], bins=bins)
    bincenters = 0.5 * (boundaries[:-1] + boundaries[1:])
    binlefts = boundaries[:-1]
    binrights = boundaries[1:]
    out = pd.DataFrame({
      "binleft": binlefts,
      "binright": binrights,
      "bincenter": bincenters,
      "count": counts,
      "selected": True })
    return(out)



def update_selection_from_hist(df, hist_df):
    conn = sqlite3.connect(":memory:")
    hist_df.to_sql("hist", conn, index = False)
    df.to_sql("plot", conn, index = False)
    query = """select title, lat, lon, views, log_views, selected
               from plot, hist
               where plot.log_views between hist.binleft and hist.binright
               and hist.selected = True
            """
    out = pd.read_sql_query(query, conn)
    out.selected = out.selected.astype("bool")
    return(out)

In [335]:
df = get_pagelist_around_location(init_loc["lat"], init_loc["lon"]).join(get_viewcounts(pages_df.index))
df = df.loc[ df.views > 0 ] # both less wasteful and we avoid trouble with log and the histogram
df["log_views"] = np.log2(df.views)
hist_df = histogram_df(df)

# Plot

In [282]:
colorscale = [
    (.00, "#0187c2"),
    (.46, "#5837ff"),
    (.58, "#8f50dc"),
    (.75, "#b162ae"),
    (.84, "#ff7674"),
    (.95, "#ffaf72"),
    (1.0, "#fff96b")
]
dash_bgcolor = "rgba(100,100,100, .8"

In [363]:
last_trigger_location = init_loc
current_location = last_trigger_location
n_clicks_old = 0 # mechanism for the button to trigger a callback

app = JupyterDash(__name__)

app.layout = html.Div([
    
    # Map background
    html.Div(
        style = {
            "width": "99%",
            "height": "300px"
        },
        children = [
            dcc.Graph(id="map")
        ]),
    
    # get_articles button at the top left
    html.Div(
        style = {
            "position": "fixed",
            "left": "15px",
            "top": "15px",
        },
        children = [
            html.Button('Was ist hier?',
                        id = 'get_article_views',
                        n_clicks = 0)
        ]
    ),
    
    # sidebar right
    html.Div(
        id = "sidebar",
        style = {
            "position": "fixed",
            "width": "20%",
            "right": "0px",
            "top": "15px",
            "marginRight": "30px",
            "color": "white"
        },
        children = [            
            html.Div(
                id = "hist-plot",
                style = {
                    "backgroundColor": dash_bgcolor,
                    "padding": "15px 15px 15px 15px",
                    "borderRadius": "5px",
                    "marginTop": "15px"
                },
                children = [
                    dcc.Graph(id = "histogram"),
                    dcc.RangeSlider(id = "slider",
                        min = min(hist_df.binleft)-.3,
                        max = max(hist_df.binright)+.3,
                        step = .001,
                        value = [min(hist_df.binleft), max(hist_df.binright)],
                        marks = {} )
                                    ]
            ),
            
            html.Div(
                id = "displayA",
                style = {
                    "backgroundColor": dash_bgcolor,
                    "padding": "15px 15px 15px 15px",
                    "borderRadius": "5px",
                    "marginTop": "15px"
                }),
            
            html.Div(
                id = "displayB",
                style = {
                    "backgroundColor": dash_bgcolor,
                    "padding": "15px 15px 15px 15px",
                    "borderRadius": "5px",
                    "marginTop": "15px"
                })
            
            
        ])
    
    #html.Div(
    #    style = {
    #        "position": "fixed",
    #        "width": "250px",
    #        "backgroundColor": "rgba(255,255,255, .3)",
    #        "right": "0px",
    #        
    #)
])

@app.callback(
    Output("map", "figure"),
    Output("histogram", "figure"),
    Output("displayA", "children"),
    Output("displayB", "children"),
    [Input('slider', 'value')],
    Input("map", "relayoutData"),
    Input("map", "clickData"),
    Input("get_article_views", "n_clicks"),
)
def update_app(slider, relayout, click, n_clicks):
    
    if relayout != {"autosize": True}:
        current_location = dict(
            lat = relayout.get("mapbox.center").get("lat"),
            lon = relayout.get("mapbox.center").get("lon")
        )
    triggered = n_clicks != n_clicks_old
    
    if triggered:
        relayout = "triggered"
        n_clicks_old = n_clicks
#        df = get_pagelist_around_location(lat = current_location["lat"], lon = current_location["lon"])
#        df = df.join(get_viewcounts(pages_df.index))
#        df = df.loc[ df.views > 0 ]
#        df["log_views"] = np.log2(df.views)
#        hist_df = histogram_df(df)
    
    # Histogram
    ### updating:
    lower, upper = slider
    hist_df.selected = (hist_df.binleft >= lower) & (hist_df.binright <= upper)
    ### rendering:
    hist = px.bar(hist_df,
        x = "bincenter",
        y = "count",
        color = hist_df.bincenter,
        color_continuous_scale = colorscale,
        opacity = list(map(opacity, hist_df.selected)),
        template = "plotly_dark",
        height = 150
    )
    hist.update_layout(
        margin = dict(t=0, r=0, b=0, l=0),
        xaxis = dict(
            title = None,
            tickvals = []
        ),
        yaxis = dict(
            title = None,
            tickvals = []
        ),
        coloraxis_showscale=False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgba(0,0,0,0)",
        bargap = 0
    )
    hist_hover = pd.DataFrame( [np.round(np.exp2(binlefts), 0),
                                np.round(np.exp2(binrights), 0)] ).transpose()
    hist.update_traces(
        marker_line_width = 0,
        customdata = hist_hover,
        hovertemplate = "%{customdata[0]}-%{customdata[1]} Aufrufe: %{y} versch. Orte"
    )

    
    
    # Map
    ### update data
    plot_df = update_selection_from_hist(df, hist_df)
    ### rendering:
    fig = px.scatter_mapbox(plot_df,
                            lat = "lat",
                            lon = "lon",
                            color = "log_views",
                            color_continuous_scale = colorscale,
                            size = "views",
                            hover_name = "title",
                            hover_data = ["views"],
                            mapbox_style="carto-darkmatter",
                            zoom = 14)
    fig.update_layout(margin = dict(t=0, r=0, b=0, l=0),
                      height = 800,
                      coloraxis_showscale=False)
    fig.update_traces(marker_sizemode = "area",
                      marker_sizeref = 5,
                      marker_sizemin = 3)
    fig.update_geos(fitbounds = False)
    fig["data"][0]["hovertemplate"] = "<b>%{hovertext}</b><br><br>Aufrufe in den letzten 30 Tagen: %{marker.size}<extra></extra>"
    fig['layout']['uirevision'] = 'something' # sort of keep zoom/position on data changes
    
    
    
    
    return fig, hist, f"{n_clicks}", f"{relayout}"


if __name__ == '__main__':
    app.run_server(mode = "inline")


In [365]:
n_clicks_old

0